# <center> Projet :  Fruits </center>



<img src="https://img.passeportsante.net/1200x675/2020-01-07/i93188-.webp" width=492>


### <center> Objectif : Déployer un modèle dans le cloud</center>
### <center>Lien projet : https://openclassrooms.com/fr/paths/164/projects/633/assignment<center>

# Initialisation

In [1]:
# SI cloud alors spark démarre ici

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
2,application_1679323912387_0003,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
try :
    %%info
except :
    pass

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
invalid syntax (<stdin>, line 2)
  File "<stdin>", line 2
    %%info
    ^
SyntaxError: invalid syntax



In [3]:
# Librairies
##Data management
import pandas as pd
import numpy as np
import io
import os
import tensorflow as tf
from PIL import Image
import time
start_time = time.time() # temps de départ



##ML
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras import Model
from pyspark.sql.functions import col, pandas_udf, PandasUDFType, element_at, split

##Parallélisation
from pyspark.sql.functions import col, pandas_udf, PandasUDFType, element_at, split
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler, StandardScaler, PCA
from pyspark.sql.functions import element_at, split, col, pandas_udf, PandasUDFType, udf
from pyspark.ml.linalg import Vectors, VectorUDT, DenseVector
from pyspark.sql.types import ArrayType, FloatType

#Connexion à S3 
import subprocess
import boto3


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
# Metavariables
CLOUD=1 # Cette variable permet de lancer le notebook dans le Cloud si égale à 1 sinon en local 0

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
# Création des chemins
if(CLOUD==1):
    PATH = "s3://p8-oc-fruits-final"
    # ESSAYER DE MIEUX ORGANISER REPERTOIRE DATA/RESULTAT
    PATH_Data = PATH+"/Data/inputs/"
    PATH_Result_parquet = PATH+"/Data/Results_parquet/"
    PATH_Result_csv = PATH+"/Data/Results_csv/"
    print('PATH:        '+\
      PATH+'\nPATH_Data:   '+\
      PATH_Data+'\nPATH_Result_parquet:   '+\
      PATH_Result_parquet+'\nPATH_Result_csv: '+PATH_Result_csv)
else :
    PATH = os.getcwd() # Chemin actuel
    PATH_Data = PATH+'/data/Test' # Chemin données
    PATH_Result = PATH+'/data/Results' # Chemin résultat   
    print('PATH:        '+\
      PATH+'\nPATH_Data:   '+\
      PATH_Data+'\nPATH_Result: '+PATH_Result)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

PATH:        s3://p8-oc-fruits-final
PATH_Data:   s3://p8-oc-fruits-final/Data/inputs/
PATH_Result_parquet:   s3://p8-oc-fruits-final/Data/Results_parquet/
PATH_Result_csv: s3://p8-oc-fruits-final/Data/Results_csv/

In [6]:
# Création d'une session spark si local

if(CLOUD==0) :
    spark = (SparkSession
                 .builder
                 .appName('P8')
                 .master('local')
                 .config("spark.sql.parquet.writeLegacyFormat", 'true')
                 .getOrCreate()
    )
    sc = spark.sparkContext
    spark
else :
    pass # car sur le cloud le kernel est pyspark

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
# Chargement des images étape 1
images = spark.read.format("binaryFile") \
  .option("pathGlobFilter", "*.jpg") \
  .option("recursiveFileLookup", "true") \
  .load(PATH_Data)

# Affichage du nombre d'images
num_images = images.count()
print("Nombre d'images chargées : ", num_images)


images.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Nombre d'images chargées :  1461
+--------------------+-------------------+------+--------------------+
|                path|   modificationTime|length|             content|
+--------------------+-------------------+------+--------------------+
|s3://p8-oc-fruits...|2023-03-19 00:33:56|  5676|[FF D8 FF E0 00 1...|
|s3://p8-oc-fruits...|2023-03-19 00:32:38|  5667|[FF D8 FF E0 00 1...|
|s3://p8-oc-fruits...|2023-03-19 00:32:40|  5660|[FF D8 FF E0 00 1...|
|s3://p8-oc-fruits...|2023-03-19 00:38:20|  5659|[FF D8 FF E0 00 1...|
|s3://p8-oc-fruits...|2023-03-19 00:33:23|  5642|[FF D8 FF E0 00 1...|
+--------------------+-------------------+------+--------------------+
only showing top 5 rows

In [8]:
# Chargement des images étape 2
images = images.withColumn('label', element_at(split(images['path'], '/'),-2))
print(images.printSchema())
print(images.select('path','label').show(5,False))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- path: string (nullable = true)
 |-- modificationTime: timestamp (nullable = true)
 |-- length: long (nullable = true)
 |-- content: binary (nullable = true)
 |-- label: string (nullable = true)

None
+---------------------------------------------------------------+----------+
|path                                                           |label     |
+---------------------------------------------------------------+----------+
|s3://p8-oc-fruits-final/Data/inputs/Test/Strawberry/15_100.jpg |Strawberry|
|s3://p8-oc-fruits-final/Data/inputs/Test/Strawberry/1_100.jpg  |Strawberry|
|s3://p8-oc-fruits-final/Data/inputs/Test/Strawberry/12_100.jpg |Strawberry|
|s3://p8-oc-fruits-final/Data/inputs/Test/Orange/r_173_100.jpg  |Orange    |
|s3://p8-oc-fruits-final/Data/inputs/Test/Strawberry/307_100.jpg|Strawberry|
+---------------------------------------------------------------+----------+
only showing top 5 rows

None

In [9]:
# Modèle d'extraction des features
model = MobileNetV2(weights='imagenet',
                    include_top=True,
                    input_shape=(224, 224, 3))
new_model = Model(inputs=model.input,
                  outputs=model.layers[-2].output)
brodcast_weights = sc.broadcast(new_model.get_weights())
new_model.summary()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu (ReLU)               (None, 112, 112, 32) 0           bn_Conv1[0][0]                   
______________________________________________________________________________________________

In [10]:
# Fonction pour l'extraction des features
def model_fn():
    """
    Retourne un modèle MobileNetV2 avec la dernière étape retirée.
    """
    model = MobileNetV2(weights='imagenet',
                        include_top=True,
                        input_shape=(224, 224, 3))
    for layer in model.layers:
        layer.trainable = False
    new_model = Model(inputs=model.input,
                  outputs=model.layers[-2].output)
    new_model.set_weights(brodcast_weights.value)
    return new_model

def preprocess(content):
    """
    Preprocesses raw image bytes for prediction.
    """
    img = Image.open(io.BytesIO(content)).resize([224, 224])
    arr = img_to_array(img)
    return preprocess_input(arr)

def featurize_series(model, content_series):
    """
    Featurize a pd.Series of raw images using the input model.
    :return: a pd.Series of image features
    """
    input = np.stack(content_series.map(preprocess))
    preds = model.predict(input)
    # For some layers, output features will be multi-dimensional tensors.
    # We flatten the feature tensors to vectors for easier storage in Spark DataFrames.
    output = [p.flatten() for p in preds]
    return pd.Series(output)

@pandas_udf('array<float>', PandasUDFType.SCALAR_ITER)
def featurize_udf(content_series_iter):
    '''
    This method is a Scalar Iterator pandas UDF wrapping our featurization function.
    The decorator specifies that this returns a Spark DataFrame column of type ArrayType(FloatType).

    :param content_series_iter: This argument is an iterator over batches of data, where each batch
                              is a pandas Series of image data.
    '''
    # With Scalar Iterator pandas UDFs, we can load the model once and then re-use it
    # for multiple data batches.  This amortizes the overhead of loading big models.
    model = model_fn()
    for content_series in content_series_iter:
        yield featurize_series(model, content_series)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

/usr/lib/spark/python/lib/pyspark.zip/pyspark/sql/pandas/functions.py:392: UserWarning: In Python 3.6+ and Spark 3.0+, it is preferred to specify type hints for pandas UDF instead of specifying pandas UDF type which will be deprecated in the future releases. See SPARK-28264 for more details.

In [11]:
# Extraction des features des  images
features_df = images.repartition(10).select(col("path"),
                                            col("label"),
                                            featurize_udf("content").alias("features")
                                           )
features_df.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+----------+--------------------+
|                path|     label|            features|
+--------------------+----------+--------------------+
|s3://p8-oc-fruits...|Strawberry|[1.5420215, 0.0, ...|
|s3://p8-oc-fruits...|Strawberry|[1.229236, 0.0014...|
|s3://p8-oc-fruits...|Strawberry|[1.3845752, 0.0, ...|
|s3://p8-oc-fruits...|    Orange|[0.13649368, 0.00...|
|s3://p8-oc-fruits...|    Orange|[0.0, 0.0, 0.0, 0...|
+--------------------+----------+--------------------+
only showing top 5 rows

# Mise en place de l'ACP

In [12]:
# Ajout de l'ACP

## 1/Standardisation
transform_vecteur_dense = udf(lambda r: Vectors.dense(r), VectorUDT())
df = features_df.withColumn('features_vecteurs', transform_vecteur_dense('features'))
scaler_std = StandardScaler(inputCol="features_vecteurs", outputCol="features_scaled", withStd=True, withMean=True)
model_std = scaler_std.fit(df)

scaler = StandardScaler(
    inputCol = 'features_vecteurs', 
    outputCol = 'features_scaled',
    withMean = True,
    withStd = True
).fit(df)

df_scaled = scaler.transform(df)

## 2/PCA
n_components = 20
pca = PCA(
    k = n_components, 
    inputCol = 'features_scaled', 
    outputCol = 'features_pca'
).fit(df_scaled)

df_pca = pca.transform(df_scaled)
vector_to_array = udf(lambda v: v.toArray().tolist(), ArrayType(FloatType())) # FONCTION UDF
df_pca = df_pca.withColumn("features_pca_array", vector_to_array("features_pca"))
df_pca=df_pca.select('path','label','features','features_pca_array')
df_pca.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+----------+--------------------+--------------------+
|                path|     label|            features|  features_pca_array|
+--------------------+----------+--------------------+--------------------+
|s3://p8-oc-fruits...|Strawberry|[1.5420215, 0.0, ...|[9.97648, 10.2693...|
|s3://p8-oc-fruits...|Strawberry|[1.229236, 0.0014...|[8.817998, 12.711...|
|s3://p8-oc-fruits...|Strawberry|[1.3845752, 0.0, ...|[9.905997, 10.955...|
|s3://p8-oc-fruits...|    Orange|[0.13649368, 0.00...|[10.980439, -8.95...|
|s3://p8-oc-fruits...|    Orange|[0.0, 0.0, 0.0, 0...|[11.21271, -7.072...|
+--------------------+----------+--------------------+--------------------+
only showing top 5 rows

In [13]:
# Afficher la variance expliquée
variance_explained = pca.explainedVariance
variance_explained.sum()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0.7083849992753264

# Enregistrement des résultats

In [14]:
if(CLOUD==1):
    # Save Parquet
    df_pca.write.mode("overwrite").parquet(PATH_Result_parquet)
    pandas_df = df_pca.toPandas()
    # Définir les informations d'identification AWS
    aws_access_key_id = '#######'
    aws_secret_access_key = '######'
    region_name = 'eu-west-3'
    # Commande pour configurer les informations d'identification AWS
    configure_cmd = f"aws configure set aws_access_key_id {aws_access_key_id} && aws configure set aws_secret_access_key {aws_secret_access_key} && aws configure set region {region_name}"
    # Exécuter la commande pour configurer les informations d'identification AWS
    subprocess.call(configure_cmd, shell=True)
    # Commande pour lister les buckets S3
    list_buckets_cmd = "aws s3 ls"
    # Exécuter la commande pour lister les buckets S3
    output = subprocess.check_output(list_buckets_cmd, shell=True)
    #print(output.decode('utf-8'))
    # Créer un client S3
    s3 = boto3.client('s3', aws_access_key_id=aws_access_key_id, aws_secret_access_key=aws_secret_access_key, region_name=region_name)
    # Spécifier le nom du bucket et le sous-répertoire pour le fichier
    bucket_name = 'p8-oc-fruits-final'
    prefix = 'Data/Results_csv/'
    # Spécifier le nom du fichier de sortie sur S3
    output_file_name = 'CSV_final_cloud.csv'
    # Convertir le DataFrame Pandas en une chaîne de caractères au format CSV
    csv_buffer = pandas_df.to_csv(index=False)
    # Télécharger le fichier dans le bucket S3
    response = s3.put_object(Body=csv_buffer, Bucket=bucket_name, Key=prefix+output_file_name)
    # Afficher la réponse
    print(response)
    print("Première PCA :",pandas_df["features_pca_array"][0])
else :
    # Enregistrement des résultats
    #Ecriture résultat
    print(PATH_Result)
    df_pca.write.mode("overwrite").parquet(PATH_Result)
    #Lecture résultat
    df = pd.read_parquet(PATH_Result, engine='pyarrow')
    print(df.loc[0,'features'].shape)
    print(df.shape)#['values']
    print("Première PCA :",df["features_pca_array"][0])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0
{'ResponseMetadata': {'RequestId': 'NBBFJ4HRVW624A9V', 'HostId': 'fQC+PR75iWWE5gTXpRm2KreTIDTzWCtYvpK9ijB0TyRJ69rJWcLbrBYXJR9PfuwhM0FiW7tlpCG3GzpQ88LUGg==', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': 'fQC+PR75iWWE5gTXpRm2KreTIDTzWCtYvpK9ijB0TyRJ69rJWcLbrBYXJR9PfuwhM0FiW7tlpCG3GzpQ88LUGg==', 'x-amz-request-id': 'NBBFJ4HRVW624A9V', 'date': 'Mon, 20 Mar 2023 15:20:02 GMT', 'x-amz-server-side-encryption': 'AES256', 'etag': '"2b188b29ddc24617990f67b2c9636c21"', 'server': 'AmazonS3', 'content-length': '0'}, 'RetryAttempts': 0}, 'ETag': '"2b188b29ddc24617990f67b2c9636c21"', 'ServerSideEncryption': 'AES256'}
Première PCA : [9.976479530334473, 10.269373893737793, -13.066469192504883, 6.341390132904053, -3.3753113746643066, 0.15535010397434235, 0.04394005239009857, -4.308056831359863, -1.4894192218780518, -2.03023362159729, -0.6426727771759033, -4.601531982421875, -1.5510313510894775, -1.323564052581787, -5.616774082183838, 2.9251489639282227, -0.409919798374176, -2.5089478492736816,

# FIN

In [15]:
# Temps final
end_time = time.time() # temps d'arrêt
elapsed_time = end_time - start_time # temps écoulé
print("Le temps écoulé est de :", elapsed_time, "secondes")
elapsed_time/60

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Le temps écoulé est de : 287.2548985481262 secondes
4.78758164246877